# Day 7

## Part 1
- commands are prepended with `$`
- `cd` is cd
- `cd /` goes back to the top directory
- `ls` is ls
    - files are printed as: `<size> <name>`
    - directories are printed as: `dir <name>`

`Find all of the directories with a total size of at most 100000. What is the sum of the total sizes of those directories?`

- Directory sizes are counted independently so the final total can contain the same file several times

In [24]:
from utils import parse_from_file, ParseConfig

# using this parser means we get blocks of output where the first lines are the
# ls printout and then at the end there are directory changes
parser = ParseConfig('\n$ ls\n', ParseConfig('\n', str))

terminal_output = parse_from_file('day_7.txt', parser)

print(terminal_output[:3])

[['$ cd /'], ['dir lhrfs', '193233 mvsjmrtn', 'dir nwh', 'dir pjsd', 'dir qfrrtb', '31987 zzdfcs', '$ cd lhrfs'], ['197903 hzl.jdj', '42249 wsbpzmbq.hws', '$ cd ..', '$ cd nwh']]


In [23]:
def change_directory(cd_command: str, current_dir: str) -> str:
    """
    returns the updated directory
    """
    arg = cd_command.replace('$ cd ', '')  # strip of the command
    if arg == '/':
        return '/'
    elif arg == '..':
        return '/'.join(current_dir.split('/')[:-2]) + '/'
    else:
        return current_dir + arg + '/'

test_dir = '/'
print(test_dir)
test_changes = ['abc', 'def', '..', '..', 'ghi', '/']
for arg in test_changes:
    test_dir = change_directory(arg, test_dir)
    print(test_dir)


/
/abc/
/abc/def/
/abc/
/
/ghi/
/


In [14]:
def get_file_system(terminal_output: list[list[str]]) -> dict:
    """
    maps out a file system from a terminal output
    """
    flat_map = {}
    current_dir = '/'
    for line in terminal_output:
        if line.startswith('$'):
            current_dir = change_directory(line, current_dir)
            print(f'changed to: {current_dir}')
            continue
        
        # else
        if current_dir not in flat_map:
            flat_map.update({current_dir: dict()})

        # WIP

    return flat_map

print(get_file_system(terminal_output[:20]))

changed to: /
dir lhrfs
193233 mvsjmrtn
dir nwh
dir pjsd
dir qfrrtb
31987 zzdfcs
$ cd lhrfs


ValueError: too many values to unpack (expected 2)